# AWS Summit Tel Aviv 2023 - Amazon SageMaker Geospatial Capabilities
## Turkey-Syria 2023 Earthquake Demo

Shimron Schiff, Solutions Architect, AWS; Boaz Horev, Sr. Solutions Architect, AWS

<img src='images/earthquake-graphics.png' width=600 alt='Earthquakes Area' title='Earthquakes Area' />

In [3]:
import pandas as pd
from shapely.geometry import Polygon, Point
import geopandas as gpd
import leafmap.foliumap as leafmap
import boto3
import sagemaker
import sagemaker_geospatial_map

In [5]:
session = boto3.Session()
execution_role = sagemaker.get_execution_role()
sg_client = session.client(service_name="sagemaker-geospatial")

In [6]:
earthquakes_map = sagemaker_geospatial_map.create_map()
earthquakes_map.set_sagemaker_geospatial_client(sg_client)
earthquakes_map.render()

SyncWidgetMap(style={'height': '100%', 'width': '100%'}, urls={'staticAssetUrlBase': 'https://d22zkd5i0te8j2.c…

In [7]:
from scripts.turkey_earthquakes import visualize_turkey_earthquakes
visualize_turkey_earthquakes(earthquakes_map)

## Data Collection
Obtain relevant images from a data collection

In [8]:
# List available data collections as part of the SageMaker Geospacial capabilities
supported_raster_data_collection = sg_client.list_raster_data_collections()
pd.DataFrame.from_dict(supported_raster_data_collection['RasterDataCollectionSummaries']) \
    [['Name', 'Arn', 'Description']].style.set_properties(**{'text-align': 'left'})

,Name,Arn,Description
0,Copernicus DEM,arn:aws:sagemaker-geospatial:us-west-2:378778860802:raster-data-collection/public/dxxbpqwvu9041ny8,"Copernicus DEM is a Digitial Surface Model which represents the surface of the Earth including buildings, infrastructure, and vegetation.,"
1,Landsat Collection 2 Level-2 Science Products,arn:aws:sagemaker-geospatial:us-west-2:378778860802:raster-data-collection/public/gmqa64dcu2g9ayx1,"Landsat Collection 2 includes scene-based global Level-2 surface reflectance and surface temperature science products.,"
2,National Agriculture Imagery Program,arn:aws:sagemaker-geospatial:us-west-2:378778860802:raster-data-collection/public/37ndema229vwa987,"National Agriculture Imagery Program acquires aerial imagery during the agricultural growing seasons in the continental U.S.,"
3,Sentinel 2 L2A COGs,arn:aws:sagemaker-geospatial:us-west-2:378778860802:raster-data-collection/public/nmqj48dcu3g7ayw8,"Sentinel-2a and Sentinel-2b imagery, processed to Level 2A (Surface Reflectance) and converted to Cloud-Optimized GeoTIFFs"


## Area of Interest (AOI)

In [9]:
# Time Range
startTime = "2023-01-27T00:00:00Z"
startTimeEOD = "2023-01-27T23:59:59Z"
endTimeSOD = "2023-02-09T00:00:00Z"
endTime = "2023-02-09T23:59:59Z"

In [10]:
# Define the center point - 37.226°N 37.014°E
center_point = Point(37.226, 37.014)

# Calculate square corners coordinates
side_length = 2
half_side = side_length / 2
xmin = center_point.x - half_side
xmax = center_point.x + half_side
ymin = center_point.y - half_side
ymax = center_point.y + half_side

# Area of Interest
aoi = Polygon([Point(xmin,ymin), Point(xmin, ymax), Point(xmax,ymax),  Point(xmax,ymin)])
print(aoi)

POLYGON ((36.226 36.014, 36.226 38.014, 38.226 38.014, 38.226 36.014, 36.226 36.014))


In [11]:
# Save to file
s = gpd.GeoSeries(aoi.boundary, crs="EPSG:4326")
polygon_gdf = gpd.GeoDataFrame(index=[0],crs='epsg:4326', geometry=s)
polygon_gdf.to_file(filename='output/polygon.geojson', driver='GeoJSON')

In [12]:
# Visualize Area of Interest
m = leafmap.Map(center=[36.844461, 37.386475], zoom=8)
m.add_geojson("output/polygon.geojson", layer_name="Footprints")
m

In [13]:
# Query Data Collection
sentinel2_arn = "arn:aws:sagemaker-geospatial:us-west-2:378778860802:raster-data-collection/public/nmqj48dcu3g7ayw8"
search_rdc_args = {
    "Arn": sentinel2_arn,
    "RasterDataCollectionQuery": {
        "AreaOfInterest": {
            "AreaOfInterestGeometry": {
                "PolygonGeometry": {
                    "Coordinates": [
                        [
                            [xmin,ymin],
                            [xmin,ymax],
                            [xmax,ymax],
                            [xmax,ymin],
                            [xmin,ymin]
                        ]
                    ]
                }
            }
        },
        "TimeRangeFilter": {
            "StartTime": startTime,
            "EndTime": endTime,
        },
        "PropertyFilters": {
            "Properties": [{"Property": {"EoCloudCover": {"LowerBound": 0, "UpperBound": 10}}}],
            "LogicalOperator": "AND",
        },
        #"BandFilter": ["visual"],
    },
}

# Identify relevant iamge URLs using search_raster_data_collection()
tci_urls = []
data_manifests = []
while search_rdc_args.get("NextToken", True):
    search_result = sg_client.search_raster_data_collection(**search_rdc_args)
    if search_result.get("NextToken"):
        data_manifests.append(search_result)
    for item in search_result["Items"]:
        tci_url = item["Assets"]["visual"]["Href"]
        print(tci_url)
        tci_urls.append(tci_url)

    search_rdc_args["NextToken"] = search_result.get("NextToken")


https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/36/S/YE/2023/2/S2A_36SYE_20230209_0_L2A/TCI.tif
https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/BV/2023/2/S2A_37SBV_20230209_0_L2A/TCI.tif
https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/CV/2023/2/S2A_37SCV_20230209_0_L2A/TCI.tif
https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/36/S/YF/2023/2/S2A_36SYF_20230209_0_L2A/TCI.tif
https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DV/2023/2/S2A_37SDV_20230209_0_L2A/TCI.tif
https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/BA/2023/2/S2A_37SBA_20230209_0_L2A/TCI.tif
https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/CA/2023/2/S2A_37SCA_20230209_0_L2A/TCI.tif
https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/37/S/DA/2023/2/S2A_37SDA_20230209_0_L2A/TCI.tif
https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentine

In [14]:
len(tci_urls)

21

## Cloud Removal Job

In [15]:
# Print job status helper function
def printJobStatus(job_arn):
    job_details = sg_client.get_earth_observation_job(Arn=job_arn)
    {print(k,": ",v) for k, v in job_details.items() if k in ["Arn", "Status", "DurationInSeconds"]}

In [102]:
# Perform Cloud Removal
eoj_input_config = {
    "RasterDataCollectionQuery": {
        "RasterDataCollectionArn": "arn:aws:sagemaker-geospatial:us-west-2:378778860802:raster-data-collection/public/nmqj48dcu3g7ayw8",
        "AreaOfInterest": {
            "AreaOfInterestGeometry": {
                "PolygonGeometry": {
                    "Coordinates": [
                        [
                            [xmin,ymin],
                            [xmin,ymax],
                            [xmax,ymax],
                            [xmax,ymin],
                            [xmin,ymin]
                        ]
                    ]
                }
            }
        },
        "TimeRangeFilter": {
            "StartTime": endTimeSOD,
            "EndTime": endTime,
        },
        "PropertyFilters": {
            "Properties": [{"Property": {"EoCloudCover": {"LowerBound": 0, "UpperBound": 10}}}],
            "LogicalOperator": "AND",
        },
    }
}

eoj_config = {
        "CloudRemovalConfig": {
            "AlgorithmName": "INTERPOLATION",
            "InterpolationValue": "-9999",
            "TargetBands": ["red", "green", "blue", "nir", "swir16"],
        },
}

eoj_response = sg_client.start_earth_observation_job(
    Name="Turkey-earthquake-cloud-removal",
    InputConfig=eoj_input_config,
    JobConfig=eoj_config,
    ExecutionRoleArn=execution_role,
)

In [117]:
eoj_arn_cloud_removal = eoj_response["Arn"]
printJobStatus(eoj_arn_cloud_removal)

Arn :  arn:aws:sagemaker-geospatial:us-west-2:261416220256:earth-observation-job/h2hwyyng88ah
DurationInSeconds :  1880
Status :  COMPLETED


## Create before/after mosaics

In [106]:
# Create Mosaic on original images (endTime)
# Before cloud removal
eoj_config_mosaic = {"GeoMosaicConfig": {"AlgorithmName": "NEAR"}}

eoj_response_mosaic_before = sg_client.start_earth_observation_job(
    Name="Turkey-earthquake-mosaic-enddate",
    InputConfig=eoj_input_config,
    JobConfig=eoj_config_mosaic,
    ExecutionRoleArn=execution_role,
)

In [115]:
eoj_arn_mosaic_before = eoj_response_mosaic_before["Arn"]
printJobStatus()

Arn :  arn:aws:sagemaker-geospatial:us-west-2:261416220256:earth-observation-job/uj8c7hwn88f6
DurationInSeconds :  1031
Status :  IN_PROGRESS


In [118]:
# Create Mosaic on transformed images (endTime)
# After cloud removal
eoj_response_mosaic_after = sg_client.start_earth_observation_job(
    Name="Turkey-earthquake-mosaic-enddate-clouds-removed",
    InputConfig = {"PreviousEarthObservationJobArn": eoj_arn_cloud_removal},
    JobConfig=eoj_config_mosaic,
    ExecutionRoleArn=execution_role,
)

In [120]:
printJobStatus(eoj_response_mosaic_after["Arn"])

Arn :  arn:aws:sagemaker-geospatial:us-west-2:261416220256:earth-observation-job/twe8xgv74kv4
DurationInSeconds :  65
Status :  IN_PROGRESS


 ## Visualize Cloud Removal Results

In [131]:
time_range_filter = {
    "start_date": endTimeSOD,
    "end_date": endTime,
}

In [320]:
# Visualize EOJ inputs and outputs
map = sagemaker_geospatial_map.create_map({"is_raster": True})
map.set_sagemaker_geospatial_client(sg_client)
# Render the map
map.render()

SyncWidgetMap(raster={'stacSearchUrl': AnyHttpUrl('https://domain-name/raster', )}, style={'height': '100%', '…

In [322]:
# Visualize AOI
#eoj_arn_mosaic_before
aoi_layer = map.visualize_eoj_aoi(Arn=eoj_arn, config={"label": "Area of Interest"})

In [328]:
# Visualize input
input_layer = map.visualize_eoj_input(
    Arn = eoj_arn,
    config = {"label": "Input"},
    time_range_filter = time_range_filter
)

In [330]:
output_layer = map.visualize_eoj_output(
    Arn = eoj_arn_cloud_removal,
    config = {"label": "output", "preset":"trueColor"},
    time_range_filter = time_range_filter
)